In [14]:
import sqlite3
import csv

# connexion à la base de données
connexion = sqlite3.connect('db_test.db')

# objet curseur pour écrire dans la base de données
# curseur_db = connexion.cursor()

# ouvrir le fichier csv en lecture
testcsv = open('test.csv', 'r', newline='')

#lireCSV = testcsv.read()
#lireCSV = csv.reader(testcsv)

> csv.reader renvoie une itération. La sortie n'est pas une liste ou un autre type pratiquement exploitable.

## Récupération des login et mdp à partir de la sortie de csv.reader dans un tableau.

Première méthode en parcourant la sortie globale (les rangées du fichiers .csv sont comptées comme des sous-éléments de la sortie).

In [16]:
tableau_login_mdp = []
rangee_tableau = 0
for rangeeCSV in list(csv.reader(testcsv)):
    tableau_login_mdp.append([]) # nouvelle rangée
    tableau_login_mdp[rangee_tableau].extend([rangeeCSV[0], rangeeCSV[1]]) # ajout de login et mdp dans la nouvelle rangée
    rangee_tableau += 1
tableau_login_mdp

[['Lui', 'mdp_test'], ['Elle', 'test_mdp']]

Deuxième méthode, sans itérer sur les éléments-mêmes de la sortie. Essai infructueux pour le moment.

In [6]:
tableau_login_mdp = []
for rangee in range(len(list(csv.reader(testcsv)))-1):
    tableau_login_mdp.append([])
    tableau_login_mdp[rangee].extend(list(csv.reader(testcsv))[rangee][0], list(csv.reader(testcsv))[rangee][1])
tableau_login_mdp

[]

### Insertion des login et mdp dans la base de données

Avec INSERT. Je verrai ensuite UPDATE.

In [23]:
tableau_login_mdp

[['Lui', 'mdp_test'], ['Elle', 'test_mdp']]

In [22]:
for rangee in tableau_login_mdp:
    
    connexion.execute("INSERT INTO test (login, mdp) VALUES (?, ?);", (rangee[0], rangee[1]))
connexion.execute("SELECT * FROM test").fetchall()

[('Lui', 'mdp_test', 'Il aime les choux.'),
 ('Elle', 'test_mdp', "Elle n'aime pas les choux."),
 ('Lui', 'mdp_test', None),
 ('Elle', 'test_mdp', None),
 ('Lui', 'mdp_test', None),
 ('Elle', 'test_mdp', None)]

In [24]:
connexion.commit()
connexion.close()